# Database / SQL Final Project

This project will involve three related CSV files.
  * [play_list_music.csv](./play_list_music.csv)
  * [play_list_track_customers.csv](./play_list_track_customers.csv)
  * [play_list_track_buy.csv](./play_list_track_buy.csv)


Your task for this project is to build a SQLite database from these files, then perform some analytics.
This project should be broken down into the following tasks:
  1. Download and inspect the files.
  1. Design a database that is **properly normalized**.
  1. Implement your database design.
  1. Load data from files into database.
  1. Write some basic queries.

All your code should be implemented in this notebook.
Below the notebook is partitioned into markdown and code execution cells.

In the cells below, connect to your database.
Remember to update the SSO to your pawprint.

In [ ]:
import pandas as pd
import getpass
import psycopg2
import numpy as np
from psycopg2.extensions import adapt, register_adapter, AsIs
# Magic adapters for the Numpy Fun of Pandas
register_adapter(np.int64,AsIs)
register_adapter(np.float64,AsIs)
mypasswd = getpass.getpass()
username = 'jch5x8'
host = 'pgsql.dsa.lan'
database = 'dsa_student'

In [1]:
# Then connects to the DB
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

# -------------- Add Content Below

# SQLAlchemy Connection Parameters
postgres_db = {'drivername': 'postgres',
               'username': username,
               'password': mypasswd,
               'host': host,
               'database' :database}

# SQLAlchemy Engine
engine = create_engine(URL(**postgres_db), echo = True)

# init connection variable
connection = None
# using a try-except
try:
    connection = engine.connect()
except Exception as err:
    print("An error occurred trying to connect: {}".format(err))
    
# -------------- Add Content Above
del mypasswd

NameError: name 'mypasswd' is not defined

# Design a database that is _properly normalized_.

Note: You can expect up approximately ten (10) tables to be derived from three CSV files.

There is no implementation cell, the output should be an ERD or sketch.

Visit the course Canvas Site for Normalization videos. 

# Implement your database design.

Use the cells below to add your `CREATE TABLE` statements.
Add extra cells as necessary

In [ ]:
DROP TABLE jch5x8.invoice_tracks, jch5x8.invoice, jch5x8.tracks, jch5x8.album, jch5x8.customer, jch5x8.contact, jch5x8.city;

CREATE TABLE IF NOT EXISTS jch5x8.city(city_id SERIAL PRIMARY KEY, city VARCHAR(50), locale VARCHAR(50), country VARCHAR(50));
CREATE TABLE IF NOT EXISTS jch5x8.contact(contact_id SERIAL PRIMARY KEY, address VARCHAR(150), city_id INT REFERENCES jch5x8.city ON DELETE SET NULL ON UPDATE CASCADE, postal_code VARCHAR(50), phone VARCHAR(50), fax VARCHAR(50), email VARCHAR(50), city VARCHAR(150), customer_id INT);
CREATE TABLE IF NOT EXISTS jch5x8.customer(customer_id INT PRIMARY KEY, first_name VARCHAR(50), last_name VARCHAR(50), company VARCHAR(50), contact_id INT REFERENCES jch5x8.contact ON DELETE SET NULL ON UPDATE CASCADE);
CREATE TABLE IF NOT EXISTS jch5x8.album(album_id SERIAL PRIMARY KEY, album VARCHAR(150), artist VARCHAR(150), genre VARCHAR(50), track_id INT);
CREATE TABLE IF NOT EXISTS jch5x8.tracks(track_id INT, album_id INT REFERENCES jch5x8.album ON DELETE SET NULL ON UPDATE CASCADE, song VARCHAR(150), playlist VARCHAR(150), media_type VARCHAR(50), bytes INT, PRIMARY KEY(track_id, playlist));
CREATE TABLE IF NOT EXISTS jch5x8.invoice(invoice_id INT PRIMARY KEY, track_id INT REFERENCES jch5x8.tracks ON DELETE SET NULL ON UPDATE CASCADE, playlist VARCHAR(150) REFERENCES jch5x8.tracks ON DELETE SET NULL ON UPDATE CASCADE, customer_id INT REFERENCES jch5x8.customer ON DELETE SET NULL ON UPDATE CASCADE, unit_price REAL);
CREATE TABLE IF NOT EXISTS jch5x8.invoice_tracks(invoice_id INT REFERENCES jch5x8.invoice ON DELETE SET NULL ON UPDATE CASCADE, track_id INT REFERENCES jch5x8.tracks ON DELETE SET NULL ON UPDATE CASCADE);

In [ ]:
result = connection.execute("CREATE TABLE IF NOT EXISTS jch5x8.city(city_id SERIAL PRIMARY KEY, city VARCHAR(50), locale VARCHAR(50), country VARCHAR(50))")
result = connection.execute("CREATE TABLE IF NOT EXISTS jch5x8.contact(contact_id SERIAL PRIMARY KEY, address VARCHAR(150), city_id INT REFERENCES jch5x8.city ON DELETE SET NULL ON UPDATE CASCADE, postal_code VARCHAR(50), phone VARCHAR(50), fax VARCHAR(50), email VARCHAR(50), city VARCHAR(150), customer_id INT)")
result = connection.execute("CREATE TABLE IF NOT EXISTS jch5x8.customer(customer_id INT PRIMARY KEY, first_name VARCHAR(50), last_name VARCHAR(50), company VARCHAR(50), contact_id INT REFERENCES jch5x8.contact ON DELETE SET NULL ON UPDATE CASCADE)")
result = connection.execute("CREATE TABLE IF NOT EXISTS jch5x8.album(album_id SERIAL PRIMARY KEY, album VARCHAR(150), artist VARCHAR(150), genre VARCHAR(50), track_id INT)")
result = connection.execute("CREATE TABLE IF NOT EXISTS jch5x8.tracks(track_id INT, album_id INT REFERENCES jch5x8.album ON DELETE SET NULL ON UPDATE CASCADE, song VARCHAR(150), playlist VARCHAR(150), media_type VARCHAR(50), bytes INT, PRIMARY KEY(track_id, playlist))")
result = connection.execute("CREATE TABLE IF NOT EXISTS jch5x8.invoice(invoice_id INT PRIMARY KEY, track_id INT REFERENCES jch5x8.tracks ON DELETE SET NULL ON UPDATE CASCADE, customer_id INT REFERENCES jch5x8.customer ON DELETE SET NULL ON UPDATE CASCADE, playlist VARCHAR(150) REFERENCES jch5x8.tracks ON DELETE SET NULL ON UPDATE CASCADE, unit_price REAL)")
result = connection.execute("CREATE TABLE IF NOT EXISTS jch5x8.invoice_tracks(invoice_id INT REFERENCES jch5x8.invoice ON DELETE SET NULL ON UPDATE CASCADE, track_id INT REFERENCES jch5x8.tracks ON DELETE SET NULL ON UPDATE CASCADE, )")

In [ ]:
col_list = ["track_id", "song", "playlist", "media_type", "bytes"]
tracks = pd.read_csv("tracks.csv", usecols = col_list)
#city.head()

tracks.to_sql('tracks',         # Table to load
            engine,               # Engine created above
            schema = username,    # Schema where table lives
            if_exists = 'append', # If table found, replace data
            index = False,        # Ignore data frame row index
            chunksize = 50        # Load 50 records from data frame at a time
        )

# Load data from files into database.

### Use Excel or Pandas to carve the provided CSV files above into the **set of appropriate files** you need to load into your database.
   1. Example: Save File As *new_csv_name.csv*
   1. Remove unneeded columns
   1. Remove duplicate rows
   1. Save File, Navigate in JupyterHub folder view (your first JupyterHub tab)
   1. Upload file


   1. Load the CSV into your database using Python.
     




## Once Loaded
  * Write SQL to show the `COUNT(*)` from each table loaded.

#  Write some basic queries.


## List each artist and the average bytes per song.

## List average number of tracks per album for each artist.

## List the top five customers in terms of tracks purchased.

## List the top genre preference per customer.

# Save your notebook, then `File > Close and Halt`